In [ ]:
!nvidia-smi

# Installing basic toolkit

In [ ]:
!pip install -qq diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install -qq opencv-contrib-python
!pip install -qq controlnet_aux

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
import torch

# Here I have preemptively uploaded the praying cat image to get a usable link from an image host; you can replace it with your own image based on your needs
image = load_image(
    "https://ice.frostsky.com/2023/08/19/7a93c14f96e2ea8b3a4b6911fff134a1.png"
)

image

In [ ]:
import cv2
import numpy as np
from PIL import Image
from transformers import pipeline

# Extract depth information
depth_estimator = pipeline('depth-estimation')
image = depth_estimator(image)['depth']
image = np.array(image)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
control_image = Image.fromarray(image)
control_image

In [ ]:
# Load ControlNet1.1's depth model
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth")
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
	"runwayml/stable-diffusion-v1-5", controlnet=controlnet
)
pipeline.enable_model_cpu_offload()

In [ ]:
prompt = ["a praying cat, cartoon style, best quality, (8k, best quality, masterpiece:1.2)"] * 2
generator = torch.manual_seed(0)
image = pipeline(prompt, num_inference_steps=30, generator=generator, image=control_image, height = 512, width = 512).images

# image.save('image_out.png')

In [ ]:
# For displaying images
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

image_grid(image, 1, 2)